In [1]:
# 这里我们实现了ResNet 18，原论文中还讨论了更深的配置。尝试实现它们。（提示：参考论文中的表1）
# http://zh.gluon.ai/chapter_convolutional-neural-networks/resnet-gluon.html
# 尝试ResNet 34

from mxnet.gluon import nn
from mxnet import nd

class Residual(nn.Block):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        strides = 1 if same_shape else 2
        self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1,
                              strides=strides)
        self.bn1 = nn.BatchNorm()
        self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm()
        if not same_shape:
            self.conv3 = nn.Conv2D(channels, kernel_size=1,
                                  strides=strides)
    
    def forward(self, x):
        print('same in / out shape:', self.same_shape)
        print('x.shape:', x.shape)
        out = nd.relu(self.bn1(self.conv1(x)))
        print(out.shape)
        out = self.bn2(self.conv2(out))
        print(out.shape)
        if not self.same_shape:
            x = self.conv3(x)
            print('x.shape:',x.shape)
        return nd.relu(out + x)

# 构建ResNet
class ResNet34(nn.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet34, self).__init__(**kwargs)
        self.verbose = verbose
        # add name_scope on the outermost Sequential
        with self.name_scope():
            # block 1
            b1 = nn.Conv2D(64, kernel_size=7, strides=2)
            # block 2
            b2 = nn.Sequential()
            b2.add(
                nn.MaxPool2D(pool_size=3, strides=2),
                Residual(64),
                Residual(64),
                Residual(64)
            )
            # block 3
            b3 = nn.Sequential()
            b3.add(
                Residual(128, same_shape=False),
                Residual(128),
                Residual(128),
                Residual(128)
            )
            # block 4
            b4 = nn.Sequential()
            b4.add(
                Residual(256, same_shape=False),
                Residual(256),
                Residual(256),
                Residual(256),
                Residual(256),
                Residual(256)
            )
            # block 5
            b5 = nn.Sequential()
            b5.add(
                Residual(512, same_shape=False),
                Residual(512),
                Residual(512)
            )
            # block 6
            b6 = nn.Sequential()
            b6.add(
                nn.AvgPool2D(pool_size=3),
                nn.Dense(num_classes)
            )
            # chain all blocks together
            self.net = nn.Sequential()
            self.net.add(b1, b2, b3, b4, b5, b6)
    
    def forward(self, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s' % (i+1, out.shape))
        return out

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = nd.random.uniform(shape=(4, 3, 96, 96))
net = ResNet34(num_classes=10, verbose=True)
net.initialize()
y = net(x)
print(y.shape)
print(net)

Block 1 output: (4, 64, 45, 45)
same in / out shape: True
x.shape: (4, 64, 22, 22)
(4, 64, 22, 22)
(4, 64, 22, 22)
same in / out shape: True
x.shape: (4, 64, 22, 22)
(4, 64, 22, 22)
(4, 64, 22, 22)
same in / out shape: True
x.shape: (4, 64, 22, 22)
(4, 64, 22, 22)
(4, 64, 22, 22)
Block 2 output: (4, 64, 22, 22)
same in / out shape: False
x.shape: (4, 64, 22, 22)
(4, 128, 11, 11)
(4, 128, 11, 11)
x.shape: (4, 128, 11, 11)
same in / out shape: True
x.shape: (4, 128, 11, 11)
(4, 128, 11, 11)
(4, 128, 11, 11)
same in / out shape: True
x.shape: (4, 128, 11, 11)
(4, 128, 11, 11)
(4, 128, 11, 11)
same in / out shape: True
x.shape: (4, 128, 11, 11)
(4, 128, 11, 11)
(4, 128, 11, 11)
Block 3 output: (4, 128, 11, 11)
same in / out shape: False
x.shape: (4, 128, 11, 11)
(4, 256, 6, 6)
(4, 256, 6, 6)
x.shape: (4, 256, 6, 6)
same in / out shape: True
x.shape: (4, 256, 6, 6)
(4, 256, 6, 6)
(4, 256, 6, 6)
same in / out shape: True
x.shape: (4, 256, 6, 6)
(4, 256, 6, 6)
(4, 256, 6, 6)
same in / out sha

In [3]:
# 读取数据
import sys
sys.path.append('..')
import utils
from mxnet import autograd
from mxnet import gluon
from mxnet import nd
from mxnet import image
from mxnet import init

batch_size = 64
resize=224

ctx = utils.try_gpu()
net = ResNet34(10, verbose=False)
net.initialize(ctx=ctx, init=init.Xavier())

def transform(data, label):
#     print(data.shape)   # (28, 28, 1)
#     print(label.shape)    # (1,)
# change data from batch x height x width x channel
# to batch x channel x height x width
#     return nd.transpose(data.astype('float32'), (0,3,1,2))/255, label.astype('float32')
    data = image.imresize(data, resize, resize)
#     data = new_data
            
    return nd.transpose(data.astype('float32'), (2,0,1))/255, label.astype('float32')
#     return data.astype('float32') / 255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

In [4]:
# 训练
import time
from mxnet import gluon

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {
    'learning_rate': 0.05
})

for epoch in range(10):
    time_start = time.time()
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
    test_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %.4f, Train acc %.4f, Test acc %.4f, Time %.0f sec" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc, time.time() - time_start))

Epoch 0. Loss: 0.6191, Train acc 0.8122, Test acc 0.8850, Time 266 sec
Epoch 1. Loss: 0.2681, Train acc 0.9011, Test acc 0.9094, Time 258 sec
Epoch 2. Loss: 0.2110, Train acc 0.9226, Test acc 0.9171, Time 258 sec
Epoch 3. Loss: 0.1705, Train acc 0.9378, Test acc 0.9243, Time 259 sec
Epoch 4. Loss: 0.1362, Train acc 0.9495, Test acc 0.9298, Time 258 sec
Epoch 5. Loss: 0.1034, Train acc 0.9629, Test acc 0.9288, Time 258 sec
Epoch 6. Loss: 0.0729, Train acc 0.9738, Test acc 0.9247, Time 258 sec
Epoch 7. Loss: 0.0512, Train acc 0.9818, Test acc 0.9317, Time 258 sec
Epoch 8. Loss: 0.0371, Train acc 0.9870, Test acc 0.9312, Time 258 sec
Epoch 9. Loss: 0.0290, Train acc 0.9899, Test acc 0.9212, Time 258 sec
